In [1]:
import pandas as pd
import numpy as np
import tempfile
import os.path
import calendar

# Ouverture du dataset avec remplacements des données manquantes afin qu'elles soient detectable par pandas en tant que Na/NaN

In [2]:
missing_values = ["n/a", "na", "--", "-"]
#file_path = "D:\_IMSD\Data viz'\dataset.csv"
file_path = "/home/sam/Documents/cours/DataViz/project_dataviz2_imsd-master/dataset.csv"
dataset = pd.read_csv(file_path, encoding="ISO-8859-1", na_values = missing_values)

/home/sam/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# copy of DataFrame => cleaned one
df = dataset.copy()
df.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [4]:
pd.set_option('display.max_columns', None)
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181691 entries, 0 to 181690
Columns: 135 entries, eventid to related
dtypes: float64(55), int64(22), object(58)
memory usage: 187.1+ MB


(181691, 135)

# Suppression des attaques n'ayant pas de date complète

#### Suppr
ession des lignes n'ayant pas de mois ou de jours (~ 0.50%)

In [5]:
print((df[df['imonth'] == 0].shape[0] + df[df['iday'] == 0].shape[0]) / df.shape[0] * 100.)
#print((891/181691)*100)

0.5014007298105024


In [6]:
df.drop(df[(df.imonth == 0) | (df.iday ==0)].index, inplace=True)

In [80]:
#df.info()
df.shape[0]

180756

Suppresion des villes 'unknown'

In [79]:
#df[df.city == 'unknown'].size
df.drop(df[(df.city == 'unknown')].index, inplace=True)

# Remplacer les numéros de mois & jours par leur abbréviation

In [8]:
df['imonth'] = df['imonth'].apply(lambda x: calendar.month_abbr[x])

# Renomer les colonnes


In [9]:
df.rename(columns={'iyear':'year','imonth':'month','iday':'day','country':'country_code', 'country_txt':'country','region_txt':'region',
                     'crit1':'criteria','attacktype1_txt':'attacktype','targtype1_txt':'targtype',
                     'targsubtype1_txt':'targsubtype','corp1':'targeted_entity','natlty1_txt':'targeted_nationality',
                     'gname':'perpetrator_group','gsubname':'perpetrator_details','claimmode_txt':'claim_mode',
                     'compclaim':'competing_claim','weaptype1_txt':'weapon_type','weapsubtype1_txt':'weapon_subtype',
                     'weapdetail':'weapon_details','nwound':'nb_vitcims_injured','nwoundte':'nb_perpetrators_injured',
                     'propextent_txt':'property_damage','ransomamt':'amount_ransom','hostkidoutcome_txt':'hostkidoutcome'
                    },inplace=True)

In [10]:
df.head()


,eventid,year,month,day,approxdate,extended,resolution,country_code,country,region,region,provstate,city,latitude,longitude,specificity,vicinity,location,summary,criteria,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype,targsubtype1,targsubtype,targeted_entity,target1,natlty1,targeted_nationality,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,targtype3,targtype3_txt,targsubtype3,targsubtype3_txt,corp3,target3,natlty3,natlty3_txt,perpetrator_group,perpetrator_details,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claim_mode,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,competing_claim,weaptype1,weapon_type,weapsubtype1,weapon_subtype,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapon_details,nkill,nkillus,nkillter,nb_vitcims_injured,nwoundus,nb_perpetrators_injured,property,propextent,property_damage,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,amount_ransom,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,Jul,2,NaN,0,NaN,58,Dominican Republic,2,Central America & Caribbean,NaN,Santo Domingo,18.456792,-69.951164,1.0,0,NaN,NaN,1,1,1,0.0,NaN,NaN,0.0,1,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,68.0,Named Civilian,NaN,Julio Guzman,58.0,Dominican Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MANO-D,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
5,197001010002,1970,Jan,1,NaN,0,NaN,217,United States,1,North America,Illinois,Cairo,37.005105,-89.176269,1.0,0,NaN,1/1/1970: Unknown African American assailants ...,1,1,1,0.0,NaN,NaN,0.0,1,0,2,Armed Assault,NaN,NaN,NaN,NaN,3,Police,22.0,"Police Building (headquarters, station, school)",Cairo Police Department,Cairo Police Headquarters,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Black Nationalists,NaN,NaN,NaN,NaN,NaN,To protest the Cairo Illinois Police Deparment,0.0,NaN,NaN,0,-99.0,-99.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Firearms,5.0,Unknown Gun Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Several gunshots were fired.,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0,Minor (likely < $1 million),NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"The Cairo Chief of Police, William Petersen, r...","""Police Chief Quits,"" Washington Post, January...","""Cairo Police Chief Quits; Decries Local 'Mili...","Christopher Hewitt, ""Political Violence and Te...",Hewitt Project,-9,-9,0,-9,NaN
6,197001020001,1970,Jan,2,NaN,0,NaN,218,Uruguay,3,South America,Montevideo,Montevideo,-34.891151,-56.187214,1.0,0,NaN,NaN,1,1,1,0.0,NaN,NaN,0.0,0,0,1,Assassination,NaN,NaN,NaN,NaN,3,Police,25.0,Police Security Forces/Officers,Uruguayan Police,Juan Maria de Lucah/Chief of Directorate of in...,218.0,Uruguay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupamaros (Uruguay),NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Firearms,2.0,Automatic or Semi-Automatic Rifle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Automatic firearm,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
7,197001020002,1970,Jan,2,NaN,0,NaN,217,United States,1,North America,California,Oakland,37.

# Création du dataset pour le random button: conserver uniquement les lignes n'ayant aucune valeur manquante des événements à compter 1997

In [11]:
#higher_1997 = df.copy()

In [12]:
#data = higher_1997[higher_1997['iyear'] >= 1997]
#data.head()
#data.info()
#data.head()

In [13]:
#random_button = data[
#    (data.iyear.notnull()) & (data.imonth.notnull()) & (data.iday.notnull()) & (data.imonth.notnull()) &
#    (data.country_txt.notnull()) & (data.city.notnull()) & (data.summary.notnull()) & (data.attacktype1_txt.notnull()) &
#    (data.targsubtype1_txt.notnull()) & (data.gname.notnull()) & (data.nkill.notnull())]

In [14]:
#random_button.info()

# Renomer les colonnes

# Suppression des colonnes non utilisées pour l'analyse

##### Pour cette analyse, nous nous baserons uniquement sur valeurs principales => Nous conserverons les données "1" de crit, attacktype_txt, targtype_txt, corp, ,target, targtype ...

In [15]:
df.drop([
    'region','specificity','vicinity','doubtterr','alternative','alternative_txt', 'attacktype1',
    'attacktype2','attacktype3', 'targtype1', 'targtype2', 'targtype3', 'targsubtype1', 'targsubtype2', 'targsubtype3',
    'natlty2', 'natlty3', 'claimmode', 'claimmode2', 'claimmode3', 'weaptype1', 'weaptype1', 'weaptype2', 'weaptype3',
    'weaptype4', 'weapsubtype1', 'weapsubtype2', 'weapsubtype3', 'weapsubtype4', 'nkillus', 'nwoundus', 'nhostkidus',
    'ransomamtus', 'hostkidoutcome', 'INT_LOG', 'INT_IDEO', 'INT_MISC', 'INT_ANY', 'related', 'crit2', 'crit3',
    'natlty1', 'attacktype2_txt', 'targtype3_txt', 'guncertain3', 'claim2', 'claimmode2_txt', 'claim3', 'claimmode3_txt',
    'attacktype2_txt','attacktype3_txt', 'targtype2_txt', 'targsubtype2_txt', 'corp2', 'target2', 'natlty2_txt', 
    'targsubtype3_txt', 'corp3','target3', 'natlty3_txt', 'gname2', 'gsubname2', 'gname3', 'gsubname3', 'guncertain2',
    'weaptype2_txt', 'weapsubtype2_txt', 'weaptype3_txt', 'weapsubtype3_txt', 'weaptype4_txt', 'weapsubtype4_txt',
    'guncertain1','propextent','ransompaidus', 'dbsource','approxdate','extended','resolution','property','property_damage',
    'propvalue','propcomment','nhours','ndays','divert','kidhijcountry','ransom','amount_ransom','ransompaid','ransomnote',
    'hostkidoutcome','nreleased'   
], axis=1, inplace=True)


In [16]:
df.shape

(180800, 44)

# Standardisation des pays

In [96]:
#df[df['city']== 'Belgrade']
# Obj : verifier si une ville a deux country differente : 
df.city.unique().size #36550
uniqueCountry = df.groupby('city')['country'].nunique()
uniqueCountry[uniqueCountry > 1].size #971
uniqueCountry[uniqueCountry > 1]
    
#print(df.groupby('city')['country'].transform('nunique'))
#df[['city', 'country']].groupby(['city']).count()
df[df['city']=='Wadi']

,eventid,year,month,day,country_code,country,provstate,city,latitude,longitude,location,summary,criteria,multiple,success,suicide,attacktype,targtype,targsubtype,targeted_entity,target1,targeted_nationality,perpetrator_group,perpetrator_details,motive,individual,nperps,nperpcap,claimed,claim_mode,competing_claim,weapon_type,weapon_subtype,weapon_details,nkill,nkillter,nb_vitcims_injured,nb_perpetrators_injured,ishostkid,nhostkid,addnotes,scite1,scite2,scite3
70526,200006080008,2000,Jun,8,92,India,Karnataka,Wadi,17.062432,76.981588,NaN,06/08/2000: Two low-intensity bombs exploded o...,1,1.0,1,0,Bombing/Explosion,Religious Figures/Institutions,Place of Worship,Roman Catholic Church,St Anne's Roman Catholic Church in Wadi,India,Hindu extremists,NaN,Unknown,0,NaN,NaN,0.0,NaN,NaN,Explosives,Unknown Explosive Type,The devices were described as low-intensity bo...,0.0,0.0,1.0,0.0,0.0,NaN,NaN,"""Four Churches Attacked With Bombs in Three In...","""India: Explosions rock Catholic church at Wad...","""Christians Blame Hindus for Bombings"", The Ne..."
124848,201312190029,2013,Dec,19,228,Yemen,Marib,Wadi,15.459284,45.429539,NaN,12/19/2013: An explosive device attached to th...,1,0.0,1,0,Bombing/Explosion,Utilities,Oil,SAFER Exploration and Production Operations Co...,Marib-Ras Isa Oil Pipeline,Yemen,Tribesmen,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,Explosives,Unknown Explosive Type,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,"""Oil pipeline in Marib sustains multiple attac...",NaN,NaN
136039,201408040026,2014,Aug,4,228,Yemen,Hadramawt,Wadi,15.914915,48.650745,NaN,08/04/2014: Assailants attacked Major General ...,1,0.0,0,0,Assassination,Military,"Military Personnel (soldiers, troops, officers...",Military of Yemen,1st Military Zone Commander: Major General Abd...,Yemen,Al-Qaida in the Arabian Peninsula (AQAP),NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,Firearms,Unknown Gun Type,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,"""Al-Qaida suspects attack Yemen military, army...","""State PAO: Yemeni Media Highlights for 6 Augu...","""Yemeni army commander survives assassinatio..."


In [92]:
for i in uniqueCountry.index:        
    if(uniqueCountry[i] > 1):
        print(df[df['city']==i][['city','country']])

         city             country
4429   Aachen  West Germany (FRG)
13111  Aachen  West Germany (FRG)
28727  Aachen  West Germany (FRG)
62851  Aachen             Germany
       city                           country
79514   Aba  Democratic Republic of the Congo
87249   Aba                           Nigeria
92294   Aba  Democratic Republic of the Congo
97556   Aba                           Nigeria
147096  Aba                           Nigeria
178844  Aba                           Nigeria
          city  country
119866  Abadam  Nigeria
152282  Abadam    Niger
156876  Abadam    Niger
167356  Abadam  Nigeria
         city      country
99669   Abobo  Ivory Coast
99855   Abobo  Ivory Coast
113787  Abobo      Nigeria
176992  Abobo  Ivory Coast
179151  Abobo  Ivory Coast
        city      country
907     Aden  South Yemen
52098   Aden        Yemen
54691   Aden        Yemen
54692   Aden        Yemen
54820   Aden        Yemen
56164   Aden        Yemen
57279   Aden        Yemen
60383   Aden      

          city    country
14293  Antigua  Guatemala
16394  Antigua  Guatemala
18558  Antigua  Nicaragua
20627  Antigua  Nicaragua
60606  Antigua  Guatemala
61574  Antigua  Guatemala
       city country
32684   Ara  Turkey
158855  Ara   India
161751  Ara   India
175193  Ara  Israel
        city country
46982  Arabe  Israel
54282  Arabe    Peru
        city       country
134519  Arar  Saudi Arabia
145420  Arar  Saudi Arabia
178313  Arar          Iraq
         city        country
34867  Arcata           Peru
68184  Arcata  United States
68185  Arcata  United States
           city   country
2561   Arequipa      Peru
11535  Arequipa      Peru
13308  Arequipa      Peru
13459  Arequipa      Peru
13460  Arequipa      Peru
14444  Arequipa      Peru
15579  Arequipa      Peru
15747  Arequipa      Peru
15994  Arequipa      Peru
15995  Arequipa      Peru
15996  Arequipa      Peru
17744  Arequipa      Peru
18355  Arequipa      Peru
18386  Arequipa      Peru
18441  Arequipa      Peru
19061  Arequipa

        city   country
112399  Bala     India
161896  Bala  Thailand
         city      country
65953  Balili      Algeria
93759  Balili  Philippines
        city      country
107092  Balo  Afghanistan
118966  Balo     Thailand
153739  Balo     Thailand
154808  Balo     Thailand
         city      country
9579    Baloi  Philippines
124329  Baloi  Philippines
158845  Baloi     Thailand
         city      country
88499   Balud  Philippines
101684  Balud        India
113024  Balud  Philippines
          city             country
2858   Bamberg  West Germany (FRG)
16348  Bamberg  West Germany (FRG)
54424  Bamberg             Germany
55874  Bamberg             Germany
         city                   country
102583  Banda                     India
137883  Banda  Central African Republic
168508  Banda                  Pakistan
         city      country
46374   Banga  Philippines
141134  Banga  Philippines
164491  Banga   Mozambique
177858  Banga  Philippines
          city                   c

        city     country
41213   Bera    Zimbabwe
97410   Bera  Bangladesh
101375  Bera  Bangladesh
165532  Bera     Nigeria
          city     country
57108   Berane  Yugoslavia
120131  Berane  Montenegro
           city  country
93889   Berhani    India
165446  Berhani  Somalia
          city             country
15      Berlin  East Germany (GDR)
34      Berlin  East Germany (GDR)
241     Berlin  East Germany (GDR)
258     Berlin  East Germany (GDR)
275     Berlin  East Germany (GDR)
453     Berlin  East Germany (GDR)
459     Berlin  East Germany (GDR)
460     Berlin  East Germany (GDR)
476     Berlin  East Germany (GDR)
507     Berlin  East Germany (GDR)
523     Berlin  East Germany (GDR)
592     Berlin  East Germany (GDR)
884     Berlin  East Germany (GDR)
886     Berlin  East Germany (GDR)
887     Berlin  East Germany (GDR)
888     Berlin  East Germany (GDR)
2645    Berlin  East Germany (GDR)
5657    Berlin  East Germany (GDR)
6398    Berlin  East Germany (GDR)
10720   Berlin     

              city    country
37585   Bongaigaon      India
37586   Bongaigaon      India
39119   Bongaigaon  Sri Lanka
67204   Bongaigaon      India
70692   Bongaigaon      India
71479   Bongaigaon      India
74681   Bongaigaon      India
75239   Bongaigaon      India
83054   Bongaigaon      India
87862   Bongaigaon      India
88319   Bongaigaon      India
88397   Bongaigaon      India
88479   Bongaigaon      India
88825   Bongaigaon      India
89258   Bongaigaon      India
92919   Bongaigaon      India
110882  Bongaigaon      India
115091  Bongaigaon      India
120247  Bongaigaon      India
120665  Bongaigaon      India
123889  Bongaigaon      India
141874  Bongaigaon      India
143324  Bongaigaon      India
         city   country
73094   Bongo    Angola
126934  Bongo  Thailand
        city             country
1575    Bonn  West Germany (FRG)
3192    Bonn  West Germany (FRG)
10932   Bonn  West Germany (FRG)
15112   Bonn  West Germany (FRG)
18246   Bonn  West Germany (FRG)
20659   Bo

...

          city     country
125434  Singra  Bangladesh
125519  Singra  Bangladesh
125520  Singra  Bangladesh
150486  Singra       India
          city   country
77489   Sinjar      Iraq
84294   Sinjar      Iraq
86469   Sinjar      Iraq
86481   Sinjar      Iraq
87128   Sinjar      Iraq
87575   Sinjar  Pakistan
88778   Sinjar      Iraq
89533   Sinjar      Iraq
89909   Sinjar      Iraq
90142   Sinjar      Iraq
90187   Sinjar      Iraq
91621   Sinjar      Iraq
91920   Sinjar      Iraq
93211   Sinjar      Iraq
93426   Sinjar      Iraq
95489   Sinjar      Iraq
100536  Sinjar      Iraq
105417  Sinjar      Iraq
127117  Sinjar      Iraq
133303  Sinjar      Iraq
135367  Sinjar      Iraq
135993  Sinjar      Iraq
135994  Sinjar      Iraq
136283  Sinjar      Iraq
139988  Sinjar      Iraq
141830  Sinjar      Iraq
141831  Sinjar      Iraq
142690  Sinjar      Iraq
142691  Sinjar      Iraq
142953  Sinjar      Iraq
145121  Sinjar      Iraq
145505  Sinjar      Iraq
145889  Sinjar      Iraq
145890  Sinjar 

         city      country
16249   Tambo         Peru
16250   Tambo         Peru
16251   Tambo         Peru
16633   Tambo         Peru
21212   Tambo         Peru
22343   Tambo         Peru
22439   Tambo         Peru
22549   Tambo         Peru
31128   Tambo         Peru
35072   Tambo         Peru
38954   Tambo         Peru
41017   Tambo         Peru
43598   Tambo         Peru
47367   Tambo         Peru
47368   Tambo         Peru
145768  Tambo  Philippines
161714  Tambo  Philippines
                  city   country
18981   Tambo district      Peru
18998   Tambo district      Peru
127365  Tambo district  Pakistan
140426  Tambo district      Peru
         city country
59777   Tanda   Egypt
63017   Tanda   Egypt
66483   Tanda   Egypt
124288  Tanda   India
         city      country
94493   Tangi  Afghanistan
97078   Tangi     Pakistan
103802  Tangi  Afghanistan
108204  Tangi  Afghanistan
114014  Tangi  Afghanistan
119901  Tangi     Pakistan
126267  Tangi     Pakistan
129793  Tangi     Pakis

                    city   country
11942  Trujillo district  Colombia
32162  Trujillo district      Peru
32163  Trujillo district      Peru
        city      country
37878   Tuba  Philippines
52481   Tuba  Philippines
125907  Tuba      Nigeria
158037  Tuba       Israel
         city      country
4848   Tudela        Spain
6380   Tudela        Spain
7411   Tudela        Spain
16934  Tudela     Colombia
17471  Tudela        Spain
35415  Tudela  Philippines
38279  Tudela        Spain
           city                   country
18461   Tulkarm                    Israel
71769   Tulkarm  West Bank and Gaza Strip
72090   Tulkarm  West Bank and Gaza Strip
168839  Tulkarm  West Bank and Gaza Strip
171243  Tulkarm  West Bank and Gaza Strip
         city   country
8754    Tulua  Colombia
8897    Tulua  Colombia
10200   Tulua  Colombia
26335   Tulua  Colombia
26339   Tulua  Colombia
26346   Tulua      Peru
29247   Tulua  Colombia
31419   Tulua  Colombia
39647   Tulua  Colombia
44596   Tulua  Colombi

          city    country
23535  Venecia  Nicaragua
43139  Venecia   Colombia
71419  Venecia   Colombia
         city        country
3817   Venice          Italy
4343   Venice          Italy
4974   Venice          Italy
4976   Venice          Italy
5125   Venice          Italy
5496   Venice          Italy
5983   Venice          Italy
6138   Venice          Italy
7125   Venice          Italy
7126   Venice          Italy
8189   Venice          Italy
8190   Venice          Italy
8191   Venice          Italy
8192   Venice          Italy
8193   Venice          Italy
8194   Venice          Italy
10115  Venice          Italy
10309  Venice          Italy
10310  Venice          Italy
10320  Venice          Italy
10321  Venice          Italy
13575  Venice          Italy
24821  Venice  United States
65351  Venice          Italy
72596  Venice          Italy
80627  Venice          Italy
            city        country
13602   Victoria         Brazil
14868   Victoria     Seychelles
14873   Victoria 

            city      country
1612    Zaragoza        Spain
2781    Zaragoza        Spain
2816    Zaragoza        Spain
9135    Zaragoza        Spain
12338   Zaragoza        Spain
16694   Zaragoza    Guatemala
19326   Zaragoza        Spain
19327   Zaragoza        Spain
21022   Zaragoza        Spain
24111   Zaragoza  Philippines
25246   Zaragoza        Spain
25662   Zaragoza     Colombia
26786   Zaragoza     Colombia
28107   Zaragoza        Spain
29614   Zaragoza     Colombia
30050   Zaragoza        Spain
30517   Zaragoza     Colombia
30518   Zaragoza     Colombia
32913   Zaragoza        Spain
33774   Zaragoza        Spain
35643   Zaragoza     Colombia
37119   Zaragoza     Colombia
37907   Zaragoza  El Salvador
38651   Zaragoza  El Salvador
38760   Zaragoza  El Salvador
41899   Zaragoza        Spain
42398   Zaragoza     Colombia
44977   Zaragoza     Colombia
45187   Zaragoza     Colombia
47069   Zaragoza     Colombia
47371   Zaragoza     Colombia
47792   Zaragoza        Spain
48714   Za

In [ ]:
new_country = {
    'Czech Republic': ['Prague', 'Usti Nad Labem', 'Halickuv Brod', 'Hradec Kralove'],
    'Slovak Republic': ['Bratislava'],
    'Serbia': ['Velki Trnovac','Medvedja','Muhovac','Pozarevac','Oraovica','Cerevajka','Bujic','Banjica',
                'Bujanovac','Lucane','Belgrade', 'Bujanovac', 'Vrbas', 'Lucani', 'Bujanovac', 'Dobrosin',
                'Suva Reka', 'Podujeva','Donje Prekaze','Subotica','Batlava','Velika Reka','Loznica','Novi Sad',
                'Cacak','Zemun','Valjevo','Nis','Surcin','Veliki Trnovac','Konculj','Borovac','Vranje'],
    'Montenegro': ['Borovica','Podgorica', 'Berane', 'Podgorica', 'Kolasin','Berane','Cetinje','Podgorica','Budva'],
    'Macedonia': ['Skopje','Careva Cesma','Prilep'],
    'Slovenia': ['Ljubljana','Stari Trg'],
    'Croatia': ['Kijevo','Dubrovnik','Osijek','Benkovac','Zagreb','Pribudic','Dvor na Uni','Knin','Polaca','Sotin',
                 'Petrinja','Titova Korenica','Pakrac','Lisani','Dolac'],
    'Kosovo': ['Streoc','Dolovo','Pristina', 'Glogovac','Titova Mitrovica','Stimlje','Decani','Peje','Kosovska Mitrovica',
                'Krpimej','Donje Ljupce','Surkis','Istinic','Bogicevica','Srbica','Runik','Prizren','Kline','Gornja Klina',
                'Djakovica','Klincina','Grabanicë','Malisevo','Opterusa','Magurë','Orlat','Grabc','Ljubenic','Blace','Voksh',
                'Dulje','Tusuz','Donji Crnobreg','Komoran','Podujevë','Prekale','Kosovo Polje','Urosevac','Besinje','Kruska'],
    'Bosnia-Herzegovina': ['Sarajevo','Odzak','Banja Luka','Mostar','Kalesija','Bosanski Brod','Bijelina','Celopek'],
    'Albania': ['Vucitrn'],
    'Russia': ['Moscow','Buynaksk','Kara-Su','Novgorod','Vasyurinskaya','Sverdlovsk','Grebenskaya','Temirgoye',
              'Irkutsk'],
    'Georgia': ['Sukhumi','Meghri district','Tbilisi'],
    'Armenia': ['Yerevan','Aygepar','Kirovakan'],
    'Azerbaijan': ['Yevlakh','Nagorno-Karabakh district','Khankendi','Baganis-Airum','Chaily','Tartar district',
                  'Stepanakert','Gyandzha','Kazakh','Kazakh district','Agstafa district','Aterk'],
    'Kyrgyzstan': ['Osh'],
    'Ukraine': ['Feodosia'],
    'Estonia': ['Tallinn','Ikla'],
    'Latvia': ['Riga','Ainazi'],
    'Lithuania': ['Lavoriskes','Birzai','Salociai','Vilnius','Medininkai']
}

In [ ]:
# Nombre de villes à renomer 
count = 0 

for i in new_country:
    if isinstance(new_country[i],list):
        count += len(new_country[i])

print(count)

In [ ]:
for countries, cities in new_country.items():
    for city in cities:
        print('Il y a en ' + countries + ' toutes les villes suivantes ' + city)

In [ ]:
'''for i in test['country']:
    if i == 'Yugoslavia':
        for countries, cities in new_country.items():
            for city in cities:
                test['country'] = countries   '''     

In [ ]:
'''for i in test.index:
    if test['country'] == 'West Germany (FRG)':
        test.at[i, 'ifor'] = 'Germany'
    elif test['country'] == 'East Germany (GDR)':
        test.at[i, 'ifor'] = 'Germany'''
        

In [ ]:
'''test[test['city'] == 'Berlin']'''

In [ ]:
'''test[test['city'] == 'Bonn']'''

In [ ]:
soviet_union = test[test['country'] == 'Soviet Union']
soviet_union['city'].unique()

In [ ]:
czechoslovakia = test[test['country'] == 'Czechoslovakia']
czechoslovakia['city'].unique()

In [ ]:
yugoslavia = test[test['country'] == 'Yugoslavia']
yugoslavia['city'].unique()

In [ ]:
df[df['perpetrator_group'] == 'Unknown']

In [ ]:
drop = df.copy()

In [ ]:
drop.dropna(axis=0, how='any',inplace=True)

In [ ]:
df.to_csv('terrorism_global.csv', sep=',', index=False, header=False, encoding='utf-8')

In [ ]:
random_button.to_csv('terrorism_random_button.csv')